In [13]:
#创建FP树节点的类
class treeNode:
    def __init__(self,nameVal,number,parentNode):
        self.name=nameVal
        self.count=number
        self.parent=parentNode
        self.children={}
        self.nodeLink=None
    
    def addCount(self,num):
        self.count+=num
        
    def disp(self,index=1):
        print '  '*index, self.name, ' ', self.count
        for child in self.children.values():
            child.disp(index+1)

In [54]:
#获取头指针表
def getHeader(dataSet,minSup):
    header={}
    for trans in dataSet:
        for item in trans:
            header[item]=header.get(item,0)+1
    for item in header.keys():
        if header[item]<minSup:
            del(header[item])
    return header
#更新树节点
def updateTree(items,inTree,header,count):
    if items[0] in inTree.children:
        inTree.children[items[0]].addCount(count)
    else:
        inTree.children[items[0]]=treeNode(items[0],count,inTree)
        #头指针表指向第一个出现的节点
        if header[items[0]][1]==None:
            header[items[0]][1]=inTree.children[items[0]]
        else:
            tileitem = header[items[0]][:]
            while (tileitem[1].nodeLink != None):
                tileitem[1] = tileitem[1].nodeLink
            tileitem[1].nodeLink = inTree.children[items[0]]
    if len(items)>1:
        updateTree(items[1:],inTree.children[items[0]],header,1)
    
#创建fp树
def createFPTree(dataSet,minSup=3):
    headerTable=getHeader(dataSet,minSup)
    reTree=treeNode('NUll Set',1,None)
    if len(headerTable.keys())==0:
        return None,None
    for k in headerTable.keys():
        headerTable[k]=[headerTable[k],None]
    #根据频率重新排序
    for trans in dataSet:
        DataDic={}
        for item in trans:
            if item in headerTable.keys():
                DataDic[item]=headerTable[item][0]
        if len(DataDic)>0:
            orderData=[v[0] for v in sorted(DataDic.items(),key=lambda p:p[1],reverse=True)]
        updateTree(orderData,reTree,headerTable,1)
    return reTree,headerTable

In [55]:
def loadSimpDat():  
    simpDat = [['r', 'z', 'h', 'j', 'p'],  
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],  
               ['z'],  
               ['r', 'x', 'n', 'o', 's'],  
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],  
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]  
    return simpDat  
simpDat = loadSimpDat()  
myFPtree, myHeaderTab = createFPTree(simpDat, 3)
myFPtree.disp()  

   NUll Set   1
     x   1
       s   1
         r   1
     z   5
       x   3
         y   3
           s   2
             t   2
           r   1
             t   1
       r   1


In [56]:
#路径归溯
def ascendTree(leafNode,prePath):
    if leafNode.parent!=None:
        prePath.append(leafNode.name)
        ascendTree(leafNode.parent,prePath)
    
#获取所有路径
def findPrePath(basePat,treeNode):
    condPath={}
    while treeNode!=None:
        prePath=[]
        ascendTree(treeNode,prePath)
        if len(prePath)>1:
            condPath[tuple(prePath[1:])]=treeNode.count
        treeNode=treeNode.nodeLink
    return condPath

In [58]:
condPath=findPrePath('r',myHeaderTab['r'][1])
print condPath
findPrePath('t',myHeaderTab['t'][1])

{('z',): 1, ('s', 'x'): 1, ('y', 'x', 'z'): 1}


{('r', 'y', 'x', 'z'): 1, ('s', 'y', 'x', 'z'): 2}

In [117]:
#创建条件FP树
def mineTree(inTree,headerTable,minSup,preFix,itemList):
    orderEle=[v[0] for v in sorted(headerTable.items(),key=lambda p:p[1])]
    for ele in orderEle:
        newFreqSet = preFix.copy() 
        newFreqSet.add(ele)
        itemList.append(newFreqSet)
        condPath=findPrePath(ele,headerTable[ele][1])
        newDataSet=[]
        for key in condPath.keys():
            for i in range(condPath[key]):
                newDataSet.append(list(key))
        reTree2,headerTable2=createFPTree(newDataSet,minSup)
        if headerTable2!=None:
            print "conditional tree for: ", newFreqSet  
            reTree2.disp(1) 
            mineTree(reTree2,headerTable2,minSup,newFreqSet,itemList)

In [118]:
freqItems=[]
mineTree(myFPtree,myHeaderTab,3,set([]),freqItems)

conditional tree for:  set(['t'])
   NUll Set   1
     y   3
       x   3
         z   3
conditional tree for:  set(['x', 't'])
   NUll Set   1
     y   3
conditional tree for:  set(['z', 't'])
   NUll Set   1
     y   3
       x   3
conditional tree for:  set(['x', 'z', 't'])
   NUll Set   1
     y   3
conditional tree for:  set(['y'])
   NUll Set   1
     x   3
       z   3
conditional tree for:  set(['y', 'z'])
   NUll Set   1
     x   3
conditional tree for:  set(['s'])
   NUll Set   1
     x   3
conditional tree for:  set(['x'])
   NUll Set   1
     z   3


In [119]:
print freqItems

[set(['r']), set(['t']), set(['x', 't']), set(['y', 'x', 't']), set(['y', 't']), set(['z', 't']), set(['y', 'z', 't']), set(['x', 'z', 't']), set(['y', 'x', 'z', 't']), set(['y']), set(['y', 'x']), set(['y', 'z']), set(['y', 'x', 'z']), set(['s']), set(['x', 's']), set(['x']), set(['x', 'z']), set(['z'])]
